# Field-Theoretic Language Model - Interactive Notebook

This notebook provides an interactive environment for exploring, training, and analyzing the field-theoretic language model. Execute cells sequentially to train and analyze the model.

In [ ]:
# Cell 1: Setup and Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import time
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from model import create_small_model, FieldTheoreticLM, FieldConfig
from data import FieldDataLoader, DatasetStats
from trainer import FieldTrainer
from inference import FieldInference
from perturbations import LevyPerturbation, BetaPerturbation, AdaptivePerturbation

# Set style
plt.style.use('dark_background')
print("🚀 Field-Theoretic Language Model - Interactive Training")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Cell 2: Visualize Architecture Components
def visualize_architecture():
    """Visualize key components of the field-theoretic architecture"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Golden spiral embedding
    ax = axes[0, 0]
    theta = np.linspace(0, 4*np.pi, 1000)
    r = 0.382 + (1-0.382) * theta / (4*np.pi)
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    ax.plot(x, y, 'gold', linewidth=2)
    ax.scatter([0], [0], c='red', s=100, zorder=5)
    ax.set_title('Golden Spiral Embedding', fontsize=14)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.2)
    
    # 2. Log-phase transformation
    ax = axes[0, 1]
    r_vals = np.linspace(0.382, 1.0, 50)
    theta_vals = np.linspace(0, 2*np.pi, 50)
    R, T = np.meshgrid(r_vals, theta_vals)
    # Log transformation
    Z = np.log(R + 1e-6)
    ax.contourf(R*np.cos(T), R*np.sin(T), Z, levels=20, cmap='viridis')
    ax.set_title('Log-Phase Transform', fontsize=14)
    ax.set_aspect('equal')
    
    # 3. Gravitational field
    ax = axes[0, 2]
    x = np.linspace(-1, 1, 50)
    y = np.linspace(-1, 1, 50)
    X, Y = np.meshgrid(x, y)
    # Gravitational potential
    r = np.sqrt(X**2 + Y**2) + 0.1
    V = -1 / (1.618 * r)
    ax.contourf(X, Y, V, levels=20, cmap='plasma')
    ax.set_title('Gravitational Potential', fontsize=14)
    ax.set_aspect('equal')
    
    # 4. Lévy flight vs Beta perturbation
    ax = axes[1, 0]
    # Simulate Lévy flight
    from scipy.stats import levy_stable
    levy_steps = [levy_stable.rvs(1.618, beta=0, scale=0.1) for _ in range(1000)]
    ax.hist(levy_steps, bins=50, alpha=0.7, color='cyan', density=True, range=(-2, 2))
    # Beta distribution
    from scipy.stats import beta
    x = np.linspace(0, 1, 1000)
    ax2 = ax.twinx()
    ax2.plot(x, beta.pdf(x, 32, 256), 'orange', linewidth=2)
    ax.set_title('Lévy (cyan) vs Beta (orange) Distributions', fontsize=14)
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')
    
    # 5. Coherence evolution
    ax = axes[1, 1]
    steps = np.arange(100)
    coherence = 0.5 + 0.4 * (1 - np.exp(-steps/20))
    ax.plot(steps, coherence, 'lime', linewidth=3)
    ax.axhline(y=0.91, color='red', linestyle='--', linewidth=2, label='Collapse Threshold')
    ax.fill_between(steps, 0, coherence, alpha=0.3, color='lime')
    ax.set_title('Coherence Evolution', fontsize=14)
    ax.set_xlabel('Evolution Steps')
    ax.set_ylabel('Coherence')
    ax.legend()
    ax.grid(True, alpha=0.2)
    
    # 6. Crystal memory formation
    ax = axes[1, 2]
    # Hebbian weight matrix
    n = 20
    W = np.random.randn(n, n)
    for i in range(50):
        pre = np.random.randn(n)
        post = np.random.randn(n)
        W += 0.01 * np.outer(post, pre)
    im = ax.imshow(W, cmap='RdBu', aspect='auto')
    ax.set_title('Crystallized Hebbian Weights', fontsize=14)
    plt.colorbar(im, ax=ax)
    
    plt.tight_layout()
    plt.show()

visualize_architecture()

In [ ]:
# Cell 3: Model Creation and Configuration
print("\n📊 Model Configurations")

# Compare model sizes
configs = {
    'small': FieldConfig(vocab_size=50257, d_model=512, n_layers=8),
    'base': FieldConfig(vocab_size=50257, d_model=768, n_layers=12),
    'large': FieldConfig(vocab_size=50257, d_model=1024, n_layers=16)
}

for name, config in configs.items():
    total_params = config.vocab_size * config.d_model + \
                  config.n_layers * config.d_model * config.d_model
    print(f"{name.capitalize()}: {total_params/1e6:.1f}M parameters")
    
# Create model for experiments
model_config = configs['small']
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"\n🔧 Creating small model on {device}...")
model_golden = create_small_model("golden").to(device)
model_logphase = create_small_model("log_phase").to(device)

print("Model architecture created successfully!")

In [ ]:
# Cell 4: Data Loading and Statistics
print("\n📚 Loading Datasets")

# Create data loaders
data_loaders = {}
for dataset_name in ["wikitext-2"]:  # Start with small dataset
    print(f"Loading {dataset_name}...")
    data_loaders[dataset_name] = FieldDataLoader(
        dataset_name, 
        batch_size=8,
        seq_length=256,
        device=device
    )
    
# Get sample batch
sample_batch = data_loaders["wikitext-2"].get_batch("train")
print(f"\nSample batch shape: {sample_batch['input_ids'].shape}")

# Compute dataset statistics
print("\nComputing token statistics...")
token_freqs = DatasetStats.compute_token_frequencies("wikitext-2", max_samples=1000)
seq_stats = DatasetStats.compute_sequence_stats("wikitext-2", max_samples=100)

print(f"Sequence stats: {seq_stats}")

# Visualize token distribution
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
top_k = 100
plt.plot(token_freqs[:top_k].numpy(), 'gold')
plt.xlabel('Token Rank')
plt.ylabel('Frequency')
plt.title('Token Frequency Distribution (Top 100)')
plt.yscale('log')
plt.grid(True, alpha=0.2)

plt.subplot(1, 2, 2)
plt.hist(token_freqs.numpy(), bins=50, color='cyan', alpha=0.7)
plt.xlabel('Frequency')
plt.ylabel('Count')
plt.title('Frequency Histogram')
plt.yscale('log')
plt.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 5: Compare Embeddings
print("\n🔍 Comparing Embedding Strategies")

# Get embeddings for sample tokens
with torch.no_grad():
    golden_embed = model_golden.embeddings(sample_batch['input_ids'][:1])
    logphase_embed = model_logphase.embeddings(sample_batch['input_ids'][:1])

# Visualize embeddings
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Golden embeddings
ax = axes[0]
im1 = ax.imshow(golden_embed[0, :, :64].cpu().T, aspect='auto', cmap='viridis')
ax.set_title('Golden Embeddings')
ax.set_xlabel('Token Position')
ax.set_ylabel('Embedding Dimension')
plt.colorbar(im1, ax=ax)

# Log-phase embeddings
ax = axes[1]
im2 = ax.imshow(logphase_embed[0, :, :64].cpu().T, aspect='auto', cmap='viridis')
ax.set_title('Log-Phase Embeddings')
ax.set_xlabel('Token Position')
ax.set_ylabel('Embedding Dimension')
plt.colorbar(im2, ax=ax)

# Difference
ax = axes[2]
diff = (logphase_embed - golden_embed)[0, :, :64].cpu().T
im3 = ax.imshow(diff, aspect='auto', cmap='RdBu', vmin=-diff.abs().max(), vmax=diff.abs().max())
ax.set_title('Difference (Log-Phase - Golden)')
ax.set_xlabel('Token Position')
ax.set_ylabel('Embedding Dimension')
plt.colorbar(im3, ax=ax)

plt.tight_layout()
plt.show()

# Compute amplification
golden_norm = torch.norm(golden_embed)
logphase_norm = torch.norm(logphase_embed)
print(f"Embedding norm ratio (log-phase/golden): {logphase_norm/golden_norm:.2f}x")

In [ ]:
# Cell 6: Test Perturbation Strategies
print("\n🌊 Testing Perturbation Strategies")

# Create perturbations
perturbations = {
    'Lévy': LevyPerturbation(scale=0.1),
    'Beta': BetaPerturbation(scale=0.1),
    'Adaptive': AdaptivePerturbation(scale=0.1)
}

# Test on sample field
test_field = golden_embed.clone()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (name, perturb) in enumerate(perturbations.items()):
    # Apply perturbation
    perturbed = perturb.perturb(test_field)
    
    # Visualize displacement
    displacement = (perturbed - test_field)[0, :, :3].cpu().numpy()
    
    # Top row: 3D trajectory
    ax = axes[i]
    ax.plot(displacement[:, 0], displacement[:, 1], 'o-', alpha=0.7)
    ax.set_title(f'{name} Perturbation (X-Y)')
    ax.set_xlabel('ΔX')
    ax.set_ylabel('ΔY')
    ax.grid(True, alpha=0.2)
    
    # Bottom row: Displacement magnitude
    ax = axes[i+3]
    disp_mag = np.linalg.norm(displacement, axis=1)
    ax.plot(disp_mag, color=['cyan', 'orange', 'lime'][i], linewidth=2)
    ax.set_title(f'{name} Displacement Magnitude')
    ax.set_xlabel('Token Position')
    ax.set_ylabel('|Δ|')
    ax.grid(True, alpha=0.2)
    
    print(f"{name}: mean displacement = {disp_mag.mean():.4f}, max = {disp_mag.max():.4f}")

plt.tight_layout()
plt.show()

In [ ]:
# Cell 7: Quick Training Test
print("\n🚀 Running Quick Training Test")

# Create small trainer for testing
test_trainer = FieldTrainer(
    model_golden,
    data_loaders["wikitext-2"],
    output_dir="test_outputs",
    log_every=10,
    eval_every=50
)

# Train for a few steps
print("Training with golden embeddings...")
test_trainer.train(num_steps=100)

# Plot training curves
metrics = test_trainer.metrics_history

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Loss
ax = axes[0, 0]
ax.plot([m.step for m in metrics], [m.loss for m in metrics], 'gold')
ax.set_xlabel('Step')
ax.set_ylabel('Loss')
ax.set_title('Training Loss')
ax.grid(True, alpha=0.2)

# Perplexity
ax = axes[0, 1]
ax.plot([m.step for m in metrics], [m.perplexity for m in metrics], 'cyan')
ax.set_xlabel('Step')
ax.set_ylabel('Perplexity')
ax.set_title('Perplexity')
ax.grid(True, alpha=0.2)

# Coherence
ax = axes[1, 0]
ax.plot([m.step for m in metrics], [m.coherence for m in metrics], 'lime')
ax.set_xlabel('Step')
ax.set_ylabel('Coherence')
ax.set_title('Field Coherence')
ax.grid(True, alpha=0.2)

# Crystal norm
ax = axes[1, 1]
ax.plot([m.step for m in metrics], [m.crystal_norm for m in metrics], 'magenta')
ax.set_xlabel('Step')
ax.set_ylabel('Norm')
ax.set_title('Crystal Weight Norm')
ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 8: Inference and Generation
print("\n💬 Testing Generation")

# Create inference engine
inference = FieldInference(model_golden, device=device)

# Test prompts
prompts = [
    "The quantum field",
    "In the beginning",
    "Once upon a time"
]

print("Generating with different perturbations:\n")

for prompt in prompts:
    print(f"Prompt: '{prompt}'")
    print("-" * 50)
    
    for ptype in ["levy", "beta", "adaptive"]:
        generated = inference.generate_text(
            prompt,
            max_length=50,
            temperature=0.8,
            perturbation_type=ptype,
            num_samples=1
        )[0]
        print(f"{ptype:>8}: {generated}")
    print()

In [ ]:
# Cell 9: Field Dynamics Visualization
print("\n🌀 Visualizing Field Dynamics")

# Analyze a sample text
sample_text = "The quantum field exhibits coherent behavior"
inference.visualize_field_evolution(sample_text)

In [ ]:
# Cell 10: Performance Profiling
print("\n⚡ Performance Profiling")

# Profile different configurations
profile_results = inference.profile_inference(
    batch_sizes=[1, 4, 8],
    seq_lengths=[128, 256]
)

# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Throughput
configs = list(profile_results.keys())
throughputs = [r['tokens_per_second'] for r in profile_results.values()]
colors = plt.cm.viridis(np.linspace(0, 1, len(configs)))

ax1.bar(range(len(configs)), throughputs, color=colors)
ax1.set_xticks(range(len(configs)))
ax1.set_xticklabels(configs, rotation=45)
ax1.set_ylabel('Tokens/Second')
ax1.set_title('Inference Throughput')
ax1.grid(True, alpha=0.2)

# Memory usage
memory = [r['memory_mb'] for r in profile_results.values()]
ax2.bar(range(len(configs)), memory, color=colors)
ax2.set_xticks(range(len(configs)))
ax2.set_xticklabels(configs, rotation=45)
ax2.set_ylabel('Memory (MB)')
ax2.set_title('GPU Memory Usage')
ax2.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 11: Summary and Next Steps
print("\n✨ Field-Theoretic Language Model Summary\n")

print("Key Findings:")
print(f"1. Log-phase embeddings show {logphase_norm/golden_norm:.1f}x amplification")
print(f"2. Adaptive perturbation balances exploration and exploitation")
print(f"3. Crystal memory grows stably through Hebbian updates")
print(f"4. Coherence threshold at 0.91 triggers field collapse")
print(f"5. No backpropagation required - pure physics-based learning")

print("\nNext Steps:")
print("1. Run full training on larger datasets (wikitext-103, C4)")
print("2. Compare embedding types in ablation study")
print("3. Analyze emergent linguistic structures in crystal memory")
print("4. Scale to larger models (base, large)")
print("5. Explore multi-modal extensions (vision, audio)")

print("\n🎯 Commands for full training:")
print("python main.py train --model-size base --dataset c4 --num-steps 50000")
print("python main.py ablation embedding_study --datasets wikitext-103,c4")